In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from sklearn import preprocessing
from math import sqrt
from keras import Sequential
from keras.layers.core import Dense, Dropout
from keras.layers.recurrent import LSTM
from sklearn.metrics import r2_score
from matplotlib import pyplot

Using TensorFlow backend.


In [2]:
def get_data1():
    dataset = pd.read_csv('samples_to_predict2021.csv')
    df1 = pd.DataFrame(dataset)
    #df.drop(df.columns[[4]], axis=1, inplace=True) 

    return df1 

In [3]:
#load_data function configured to accept any amount of features
#https://github.com/etai83/lstm_stock_prediction
def load_data(DEC, seq_len):
    amount_of_features = len(DEC.columns)
    #data = DEC.as_matrix() 
    data = DEC.values
    sequence_length = seq_len + 1
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])

    result = np.array(result)
    row = round(0.8 * result.shape[0])
    train = result[:int(row), :]
    x_train = train[:, :-1]
    y_train = train[:, -1][:,-1]
    x_test = result[int(row):, :-1]
    y_test = result[int(row):, -1][:,-1]


    
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], amount_of_features))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], amount_of_features))  

    return [x_train, y_train, x_test, y_test]

In [4]:
# To read data
df1 = get_data1()
df1.tail()

,Y,M,D,Temp,ND,WD,EC
360,2021,12,27,13.6,458,1,0
361,2021,12,28,14.2,458,1,0
362,2021,12,29,12.3,458,1,0
363,2021,12,30,11.9,458,1,0
364,2021,12,31,11.7,458,0,0


In [5]:
window = 35
X_train, y_train, X_test, y_test = load_data(df1[::1], 35)
print("X_train", X_train.shape)
print("y_train", y_train.shape)
print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_train (263, 35, 7)
y_train (263,)
X_test (66, 35, 7)
y_test (66,)


In [6]:
from keras.models import load_model

# load model from single file
model = load_model('ECCollageMODEL.h5')
# make predictions
yhat = model.predict(X_test, verbose=0)


In [7]:
#short term pred
Year = X_test[(0+window):(30+window) , 0 , 0] 
Month = X_test[(0+window):(30+window) , 0 , 1] 
Day = X_test[(0+window):(30+window) , 0 , 2] 

#pd.to_numeric(p, downcast="float")
Actual = y_test.flatten()[0:30] 
pred_value = yhat.flatten()[0:30] *10000

pd.options.display.float_format = '{:.4f}'.format

ECP = pd.DataFrame({'Year': np.int_(Year) , 'Month': np.int_(Month), 'Day': np.int_(Day),
                    'Predicted (KWH)': pred_value 
})


ECP

,Year,Month,Day,Predicted (KWH)
0,2021,10,26,2526.4995
1,2021,10,27,2526.3696
2,2021,10,28,2526.3647
3,2021,10,29,2526.4421
4,2021,10,30,2526.4678
5,2021,10,31,2526.2756
6,2021,11,1,2526.2146
7,2021,11,2,2530.5537
8,2021,11,3,2528.9197
9,2021,11,4,2529.8572
